## Librairies

Import all the librairies required for the program

In [1]:
import pandas as pd
import datetime as dt
from pymongo import MongoClient

import gspread
from google.oauth2 import service_account
from googleapiclient.discovery import build

from gspread_dataframe import set_with_dataframe

## Configurations

Set up the several connexion required to run the notebook (Databases, API, etc.)

### MongoDB Cursor

In [ ]:
db_user_name = 'your_db_username'
db_password = 'your_db_password'
db_name = 'your_db_name'
db_host = 'your_db_host'
db_port = 'your_db_port'

In [ ]:
mg_client = MongoClient(f'mongodb://{db_user_name}:{db_password}@{db_host}:{db_port}')
mg_db = mg_client[db_name]

### Google APIs

In [ ]:
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

path_to_service_account_json = '/the/full/location/of/your/service/account/credentials.json'
sa_credentials = service_account.Credentials.from_service_account_file(path_to_service_account_json)

sa_creds = sa_credentials.with_scopes(scopes)
gw_client = gspread.authorize(credentials = sa_creds)

## Functions

Define all the functions that will be used within the program

In [4]:
def create_sheet_get_id(sheet_title, folder_id, service_account_creds):
    '''
    This function will create a new google sheet file within a specific doogle drive folder.
    Args:
        sheet_title (str): The title of the new Google Sheet.( E.g. Tuto - Python x Google Sheets)
        folder_id (str): The ID of the Google Drive folder where the sheet will be created. If the link of the drive folder is https://drive.google.com/drive/folders/1a2b3c4d5e6f7g8h9i0j, then the folder_id is 1a2b3c4d5e6f7g8h9i0j.
        service_account_creds (service_account.Credentials): The service account credentials to authenticate with Google Drive API.
    '''
    drive_api = build('drive', 'v3', credentials=service_account_creds)

    # Check if arguments are valid
    if not folder_id:
        raise ValueError("Folder ID cannot be empty.")
    if not isinstance(folder_id, str):
        raise TypeError("Folder ID must be a string.")
    folder_metadata = drive_api.files().get(fileId=folder_id, fields='id').execute()
    if not folder_metadata:
        raise ValueError(f"Folder with ID {folder_id} does not exist.")

    if not sheet_title:
        raise ValueError("Sheet title cannot be empty.")
    if not isinstance(sheet_title, str):
        raise TypeError("Sheet title must be a string.")

    if not service_account_creds:
        raise ValueError("Service account credentials cannot be empty.")
    if not isinstance(service_account_creds, service_account.Credentials):
        raise TypeError("Service account credentials must be an instance of service_account.Credentials.")

    # Create the new Google Sheet
    file_metadata = {'name': sheet_title, 'parents': [folder_id], 'mimeType': 'application/vnd.google-apps.spreadsheet',}
    newsheet = drive_api.files().create(body=file_metadata).execute()
    sheet_id = newsheet['id']
    
    return sheet_id

In [27]:
def fill_sheet_with_pandas_dataframes(sheet_id, data_frames_dict, service_account_creds):
    '''
    This function will fill a Google Sheet with data.
    Args:
        sheet_id (str): The ID of the Google Sheet to fill with data.
        data_frames_dict (dict): The data to fill the Google Sheet with. It should be a dictionary where keys are sheet names and values are pandas DataFrames.
        service_account_creds (service_account.Credentials): The service account credentials to authenticate with Google Sheets API.
    '''
    if not sheet_id:
        raise ValueError("Sheet ID cannot be empty.")
    if not isinstance(sheet_id, str):
        raise TypeError("Sheet ID must be a string.")
    if not service_account_creds:
        raise ValueError("Service account credentials cannot be empty.")
    if not isinstance(service_account_creds, service_account.Credentials):
        raise TypeError("Service account credentials must be an instance of service_account.Credentials.")
    
    # Authorize the Google Sheets API
    gw_client = gspread.authorize(credentials=service_account_creds)
    workbook = gw_client.open_by_key(sheet_id)
    for sheet_name, df in data_frames_dict.items():
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"Data for sheet '{sheet_name}' must be a pandas DataFrame.")
        
        # Clear the existing content of the sheet
        try:
            worksheet = workbook.worksheet(sheet_name)
            worksheet.clear()
        except gspread.WorksheetNotFound:
            worksheet = workbook.add_worksheet(title=sheet_name, rows="100", cols="20")
        
        # Set the DataFrame to the Google Sheet
        set_with_dataframe(worksheet, df, include_index=False, include_column_header=True)
        print(f"Data for sheet '{sheet_name}' has been written to the Google Sheet.")
    
    sheet_to_delete = workbook.worksheet("Sheet1") # Delete the default sheet created by Google Sheets
    workbook.del_worksheet(sheet_to_delete)  

In [31]:
def share_google_sheet_with_message(sheet_id, email_list, acess, service_account_creds, message=""):
    # Authenticate with service account
    drive_service = build('drive', 'v3', credentials=service_account_creds)

    for email_to_share in email_list:
        # Create permission
        permission = {
            'type': 'user',
            'role':  acess, #'reader',
            'emailAddress': email_to_share
        }

        # Share the file
        drive_service.permissions().create(
            fileId=sheet_id,
            body=permission,
            sendNotificationEmail=True,
            emailMessage=message
        ).execute()

    print(f"Google Sheet shared with {email_to_share} as {acess}.")

## Variables

Declare all the variables that will be used within the program. Here we decide to process the data on daily basis.

In [6]:
yesterday_start_datetime = dt.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - dt.timedelta(days=1)
today_start_datetime = dt.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

## Data Extraction & Processing

Extract the needed data from the database and process them to generate the needed insights

### New accounts (registration)

In [7]:
new_accounts_via_agents_ = mg_db['dim_customers'].aggregate([
    {
        '$match': {
            'created_at': {
                '$gte': yesterday_start_datetime, 
                '$lt': today_start_datetime
            }, 
            'agent_id': {
                '$ne': None
            }
        }
    }, {
        '$lookup': {
            'from': 'dim_agents', 
            'localField': 'agent_id', 
            'foreignField': 'unique_id', 
            'as': 'agents_details'
        }
    }, {
        '$unwind': {
            'path': '$agents_details', 
            'preserveNullAndEmptyArrays': True
        }
    }, {
        '$project': {
            '_id': 0, 
            'Registration_Date': '$created_at', 
            'Customer_ID': '$unique_id', 
            'Agent_ID': {'$toInt': '$agent_id'}, 
            'Agent_WalletNumber': '$agents_details.wallet_number'
        }
    }
])

new_accounts_via_agents = pd.DataFrame(list(new_accounts_via_agents_))

In [8]:
new_accounts_via_agents.head()

,Registration_Date,Customer_ID,Agent_ID,Agent_WalletNumber
0,2025-05-30 07:07:50.237,2193,62,22901200000062
1,2025-05-30 05:09:28.798,20797,22,22901200000022
2,2025-05-30 04:40:47.727,25199,32,22901200000032
3,2025-05-30 12:25:11.658,15871,98,22901200000098
4,2025-05-30 00:27:04.015,7508,30,22901200000030


In [9]:
new_accounts_via_agents['Registration_Wage'] = 2.99
registration_fees = new_accounts_via_agents[['Agent_ID', 'Agent_WalletNumber', 'Registration_Wage']].groupby(['Agent_ID', 'Agent_WalletNumber']).sum().reset_index()
registration_fees.head()

,Agent_ID,Agent_WalletNumber,Registration_Wage
0,1,22901200000001,5.98
1,2,22901200000002,8.97
2,3,22901200000003,11.96
3,4,22901200000004,2.99
4,6,22901200000006,5.98


### First deposit (new clients operation)

In [10]:
first_deposit_via_agents_ = mg_db['fct_top_ups'].aggregate([
    {
        '$match': {
            'completed_at': {
                '$gte': yesterday_start_datetime, 
                '$lt': today_start_datetime
            }, 
            'agent_id': {
                '$ne': None
            }, 
            'is_completed': True, 
            'is_first_top_up': True
        }
    }, {
        '$lookup': {
            'from': 'dim_agents', 
            'localField': 'agent_id', 
            'foreignField': 'unique_id', 
            'as': 'agents_details'
        }
    }, {
        '$unwind': {
            'path': '$agents_details', 
            'preserveNullAndEmptyArrays': True
        }
    }, {
        '$project': {
            '_id': 0, 
            'FirstDeposit_Date': '$completed_at', 
            'FirstDeposit_Amount': '$amount',
            'Customer_ID': '$customer_id', 
            'Agent_ID': {'$toInt': '$agent_id'}, 
            'Agent_WalletNumber': '$agents_details.wallet_number'
        }
    }
])

first_deposit_via_agents = pd.DataFrame(list(first_deposit_via_agents_))

In [11]:
first_deposit_via_agents.head()

,FirstDeposit_Date,FirstDeposit_Amount,Customer_ID,Agent_ID,Agent_WalletNumber
0,2025-05-30 00:30:33.399,20.0,6674,84,22901200000084
1,2025-05-30 00:53:55.822,8.0,17878,20,22901200000020
2,2025-05-30 02:24:27.985,0.4,25757,39,22901200000039
3,2025-05-30 04:57:12.219,4.0,21818,94,22901200000094
4,2025-05-30 06:04:49.662,8.0,634,60,22901200000060


In [12]:
first_deposit_via_agents['Deposit_Wage'] = 1.99
first_Deposit_fees = first_deposit_via_agents[['Agent_ID', 'Agent_WalletNumber', 'Deposit_Wage']].groupby(['Agent_ID', 'Agent_WalletNumber']).sum().reset_index()
first_Deposit_fees.head()

,Agent_ID,Agent_WalletNumber,Deposit_Wage
0,1,22901200000001,3.98
1,2,22901200000002,1.99
2,3,22901200000003,1.99
3,5,22901200000005,1.99
4,6,22901200000006,1.99


In [13]:
same_day_activities = new_accounts_via_agents[['Registration_Date', 'Customer_ID', 'Agent_ID', 'Agent_WalletNumber']].merge(
    first_deposit_via_agents[['FirstDeposit_Date', 'Customer_ID', 'Agent_ID']], 
    on=['Customer_ID', 'Agent_ID'], 
    how='inner'
)

In [14]:
same_day_activities.head()

,Registration_Date,Customer_ID,Agent_ID,Agent_WalletNumber,FirstDeposit_Date


In [15]:
same_day_activities['same_day_bonus'] = 0.99
same_day_bonus_fees = same_day_activities[['Agent_ID', 'Agent_WalletNumber', 'same_day_bonus']].groupby(['Agent_ID', 'Agent_WalletNumber']).sum().reset_index()
same_day_bonus_fees.head()

,Agent_ID,Agent_WalletNumber,same_day_bonus


In [16]:
all_fees = registration_fees.merge(
    first_Deposit_fees, 
    on=['Agent_ID', 'Agent_WalletNumber'], 
    how='outer', 
    suffixes=('_registration', '_first_deposit')
).merge(
    same_day_bonus_fees, 
    on=['Agent_ID', 'Agent_WalletNumber'], 
    how='outer'
)
all_fees.fillna(0, inplace=True)
all_fees['Total_Fees'] = all_fees['Registration_Wage'] + all_fees['Deposit_Wage'] + all_fees['same_day_bonus']

In [17]:
all_fees.head()
all_fees['Date'] = yesterday_start_datetime.date()
all_fees['Description'] = 'Congratulation! You have earned a total amount of ' + all_fees['Total_Fees'].astype(str) + f' ECO for your acquisition operations of {yesterday_start_datetime.date()} as follow; New Registration: ' + all_fees['Registration_Wage'].astype(str) + ', First Deposit: ' + all_fees['Deposit_Wage'].astype(str) + ', Same Day Bonus: ' + all_fees['same_day_bonus'].astype(str)

In [18]:
all_fees.loc[0, 'Description']

'Congratulation! You have earned a total amount of 9.96 ECO for your acquisition operations of 2025-05-30 as follow; New Registration: 5.98, First Deposit: 3.98, Same Day Bonus: 0.0'

In [19]:
payments_file = all_fees[['Agent_ID', 'Date', 'Agent_WalletNumber', 'Total_Fees', 'Description']].rename(
    columns={
        'Agent_ID': 'Agent_ID',
        'Date': 'Operation_Date', 
        'Agent_WalletNumber': 'WalletNumber', 
        'Total_Fees': 'Amount', 
        'Description': 'Payment_Description'
    }
)

In [20]:
payments_file.head()

,Agent_ID,Operation_Date,WalletNumber,Amount,Payment_Description
0,1,2025-05-30,22901200000001,9.96,Congratulation! You have earned a total amount...
1,2,2025-05-30,22901200000002,10.96,Congratulation! You have earned a total amount...
2,3,2025-05-30,22901200000003,13.95,Congratulation! You have earned a total amount...
3,4,2025-05-30,22901200000004,2.99,Congratulation! You have earned a total amount...
4,5,2025-05-30,22901200000005,1.99,Congratulation! You have earned a total amount...


## Reporting

In [21]:
# test_wb_id = create_sheet_get_id(
#     sheet_title='Tuto - Python x Google Sheets',
#     folder_id='1VuES2B8kVVw7Wfi458999hI1C50Zia5Y',  # Replace with your actual folder ID
#     service_account_creds=sa_creds
# )

### Payment file loading

In [22]:
payment_sheet_id = create_sheet_get_id(
    sheet_title=f'Agent Payments {yesterday_start_datetime.date()}',
    folder_id='1ijXSSDyYayKizn5L_ANYuwm9QNEUn9nD',  # Replace with your actual folder ID
    service_account_creds=sa_creds
)

In [28]:
fill_sheet_with_pandas_dataframes(
    sheet_id=payment_sheet_id,
    data_frames_dict={
        'Payments': payments_file
    },
    service_account_creds=sa_creds
)

Data for sheet 'Payments' has been written to the Google Sheet.


In [32]:
share_google_sheet_with_message(
    sheet_id=payment_sheet_id,
    email_list=['bola@isheero.com'],
    acess='reader',  # 'reader' or 'writer'
    service_account_creds=sa_creds,
    message=f'Hello, \n\nPlease find attached the payments for agents for {yesterday_start_datetime.date()}. \n\nBest regards,\nIntelytix Team'
)

Google Sheet shared with bola@isheero.com as reader.


In [29]:
performance_sheet_id = create_sheet_get_id(
    sheet_title=f'Agent Performance {yesterday_start_datetime.date()}',
    folder_id='1mvb_Q8fRzQ1pA60V1s1TtDYCYkDBdGR_',  # Replace with your actual folder ID
    service_account_creds=sa_creds
)

In [30]:
fill_sheet_with_pandas_dataframes(
    sheet_id=performance_sheet_id,
    data_frames_dict={
        'New Accounts': new_accounts_via_agents,
        'First Deposits': first_deposit_via_agents,
        'Same Day Activities': same_day_activities,
        'All Fees': all_fees
    },
    service_account_creds=sa_creds
)

Data for sheet 'New Accounts' has been written to the Google Sheet.
Data for sheet 'First Deposits' has been written to the Google Sheet.
Data for sheet 'Same Day Activities' has been written to the Google Sheet.
Data for sheet 'All Fees' has been written to the Google Sheet.
